# План работы:
- Кроме Videos.md должен быть файл Videos.html - для удобства просмотра YouTube роликов на компьютере. В начале страницы суммировать время и число видеороликов.
- Для оценки структуры курса и генерации элементов - ячейка с считыванием всех имеющихся блоков, в которых source для Markdown содержит знаки решетки. Это автоматически отправлять в Syllabus
- Сделать взаимодополняющимися разделы "Литература" - Переносить из файла Literature.md в соответствующие разделы и обратно

In [150]:
import os
import json
os.chdir('../course') # Все действия осуществляются из директории курса

from IPython.display import display, Markdown, Latex

# Определение путей всех имеющихся в курсе ipynb-файлов

In [151]:
ipynb_paths = []

for dirname, dirnames, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        if filename.endswith('.ipynb') & (not filename.endswith('-checkpoint.ipynb')):
            ipynb_paths.append(os.path.join(dirname, filename))

# Актуальное содержание в READ.me файле

In [152]:
def chapters():
    chs = dict()
    for i, ipynb_path in enumerate(sorted(ipynb_paths)):
        with open(ipynb_path) as f:
            data = json.load(f)
            name = data['cells'][0]['source'][0]  # Заголовок файла это всегда первая ячейка
            markdown_path = os.path.join(os.path.basename(os.path.dirname(ipynb_path)), os.path.basename(ipynb_path))
            # срез так как название начинается с решетки и пробела
            chs[ipynb_path] = '{0}. [{1}]({2})'.format(i+1, name[2:].strip(), markdown_path)
    return chs
            
with open('README.md') as f:
    blocks = f.read().split('#')
    for i, block in enumerate(blocks):
        if block.startswith(' Содержание курса'):
            blocks[i] = ' Содержание курса\n\n' + '\n'.join([chapters()[key] for key in sorted(chapters())]) + '\n\n'
    f.close()

with open('README.md', 'w') as f:           
    f.write("#".join(blocks)) 
    f.close()

print('Ниже идет текущий вид сгенерированного файла READ.me')
with open('README.md') as f:
    display(Markdown(f.read()))
    f.close()

Ниже идет текущий вид сгенерированного файла READ.me


==Обращаем внимание, что курс представлен в черновой форме, в том числе эта страница==

# Курс «Наноматериалы»

Курс «Наноматериалы» посвящен современному материаловедению низкоразмерных систем и состоит из блокнотов [Jupyter](https://jupyter.org), содержащих конспект лекции, программируемые иллюстрации теоретических положений и лабораторные работы по моделированию наноматериалов. Курс представлен пока в незаконченной, черновой форме, запуск курса — февраль 2019 г. Экспериментальные лабораторные работы и чтение лекций проводятся на [кафедре микро- и наноэлектроники](https://etu.ru/ru/fakultety/fakultet-elektroniki/sostav-fakulteta/kafedra-mikroelektroniki/) [СПбГЭТУ «ЛЭТИ»](https://etu.ru/ru/universitet/), учебно-научная лаборатория «Наноматериалы».

Выбор такого подхода обусловлен следующими причинами:
* Современное материаловедение развивается настолько динамично, что любое печатное пособие устаревает в течение нескольких лет. Поэтому необходима форма, позволяющая непрерывно обновлять курс так, чтобы он оставался доступен студентам не только текущего года, но и прошлых лет, а также выпускникам.
* Наиболее эффективным для изучения темы видится подход, при котором студент может не только читать теорию, но сразу же проверять гипотезы, закрепляя собственное понимание. Формат Jupyter Notebook позволяет сочетать в одном файле текст, формульные выражения, видео и программы, моделирующие рассматриваемые аналитические выражения.
* Тенденцией образования стало овладение людей различных профессий программированием для автоматизации различных рутинных задач. В данном курсе студенты для его прохождения не обязаны знать язык программирования , но те, кто захотят разобраться в деталях, смогут приобрести дополнительную практику. В курсе в качестве языка программирования используется Python версии 3.7.

# Как пользоваться курсом

Курс создается с возможностью чтения непосредственно с экрана компьютера, в том числе мобильных устройств. Однако для запуска программных ячеек и выполнения лабораторных работ потребуется воспользоваться одним из  вариантов:
* Воспользоваться удаленным сервисом https://mybinder.org (будет описано позже, есть ограничение в объеме выделяемой памяти)
* Сохранить каталог с курсом локально: 1) простым образом, сохранив zip-папку и обновляя ее при необходимости, 2) сохранив и обновляя при помощи среды Git. Для работы с файлами используется среда Jupyter. Самым простым способом установки большинства необходимых библиотек является дистрибутив [Anaconda](https://www.anaconda.com/download/).

# Оценка успеваемости

В конце курса проводится экзамен. Для допуска к экзамену необходимо провести и защитить отчеты по всем лабораторным работам, а также написать  все контрольные работы на удовлетворительную оценку. Контрольные работы состоят из пяти вопросов и содержат в том числе упражнения и задачи. В соответствии с баллами, полученными по результатам контрольных работы и защиты отчетов по лабораторным работам составляется рейтинг, которых учитывается при выставлении оценки на экзамене.

# Содержание курса

1. [Размерные эффекты и корреляционные радиусы физических явлений](01_-_Nanoscale/Lecture.ipynb)
2. [Электронное строение низкоразмерных структур](02_-_From_atoms_to_nanomaterials/Lecture_and_lab.ipynb)
3. [Кластеры](03_-_Clusters/Lecture.ipynb)
4. [Металлические наночастицы](04_-_Metal_NPs/Lecture.ipynb)
5. [Нанокристаллы полупроводниковых материалов](05_-_Semiconductor_NPs/Lecture.ipynb)
6. [Наночастицы широкозонных полупроводников и диэлектриков](06_-_Widegap_NPs/Lecture.ipynb)
7. [Общие свойства наночастиц различной природы](07_-_Properties_of_NPs/Lecture.ipynb)
8. [Нанопроволоки и наностержни](08_-_1D_Structures/Lecture.ipynb)
9. [Тонкие пленки и покрытия](09_-_2D_Nanomaterials/01_Thin_films.ipynb)
10. [Фотонные кристаллы и многослойные структуры](09_-_2D_Nanomaterials/02_Photonic_crystals.ipynb)
11. [Консолидированные наноматериалы и нанокомпозиты](10_-_3D_Nanomaterials/Nanocomposites.ipynb)
12. [Самоорганизация и самосборка](10_-_3D_Nanomaterials/Self-organization.ipynb)

# Предпосылки и продолжения

В качестве базовых курсов предполагаются ... В курсе почти не рассматриваются пористые материалы, углеродные материалы и органическая электроника, так как соответствующие курсы проходят у студентов направления в осеннем семестре следующего учебного года. Также малое внимание уделено направлению магнитных материалов в связи с тем, что соответствующий курс читается в бакалавриате.

==Обращаем внимание, что курс представлен в черновой форме, в том числе эта страница==

# Полная структура курса

In [153]:
# Рассматриваем только для markdown cells

# Генерация страницы с полным списком видео

Для удобства просмотра видео на досуге сделаем общую страницу, в которой видеофрагменты из всех Jupyter-блокнотов будут собираться в общий файл [Videos.md]().

In [160]:
md_first_lines = '''
# Видеотека

Данный файл содержит подборку из всех видеофрагментов
курса "Наноматериалы". Этот файл более удобен, если вы
открыли его на GitHub. Однако разметка Markdown не позволяет
встраивать видео. Каждое изображение является ссылкой на 
видеоролик. Если вы открыли курс на компьютере,
советуем воспользоваться [html-файлом](Videos.html).
Это позволит воспроизводить все видео на одной странице.

Приведенные видеофрагменты не являются обязательными
к просмотру, но позволяют увидеть рассматриваемые проблемы
с позиций других исследователей. Некоторые видеоролики
полезны как примеры лабораторных практик по синтезу
и диагностике различных форм наноматериалов.\n\n'''


page = ''

for ipynb_path in sorted(chapters().keys()):
    with open(ipynb_path) as f:
        head = '# {} \n'.format(chapters()[ipynb_path])
        page += head
        data = json.load(f)
        for cell in data['cells']:
            if (cell['cell_type'] == 'code') & ('YouTubeVideo' in ''.join(cell['source'])):
                d = "".join(cell['source'])
                d = d.replace('\n', ' ')
                try:
                    d = d[3:d[d.index("'''")+3:].index("'''")+3]
                    d = d.lstrip()
                except ValueError:
                    d = '**У этого видео пока нет описания.**'
                for line in cell['source']:
                    if "YouTubeVideo('" in line.strip():
                        short_link = line[len("YouTubeVideo('"):-len(")'")]
                image_url = 'https://img.youtube.com/vi/{}/0.jpg'.format(short_link)
                movie_url = 'https://www.youtube.com/watch?v={}'.format(short_link)
                image_block = '[![]({0})]({1})\n'.format(image_url, movie_url)
                movie_with_description = '{0}\n\n {1}\n\n ---- \n'.format(d, image_block)
                page += movie_with_description

display(Markdown(md_first_lines+page))


# Видеотека

Данный файл содержит подборку из всех видеофрагментов
курса "Наноматериалы". Этот файл более удобен, если вы
открыли его на GitHub. Однако разметка Markdown не позволяет
встраивать видео. Каждое изображение является ссылкой на 
видеоролик. Если вы открыли курс на компьютере,
советуем воспользоваться [html-файлом](Videos.html).
Это позволит воспроизводить все видео на одной странице.

Приведенные видеофрагменты не являются обязательными
к просмотру, но позволяют увидеть рассматриваемые проблемы
с позиций других исследователей. Некоторые видеоролики
полезны как примеры лабораторных практик по синтезу
и диагностике различных форм наноматериалов.

# 1. [Размерные эффекты и корреляционные радиусы физических явлений](01_-_Nanoscale/Lecture.ipynb) 
Видеоиллюстрация изменения "цвета" кубка Ликурга на экспозиции в Британском музее. 

 [![](https://img.youtube.com/vi/aTL8Gx6TlCs/0.jpg)](https://www.youtube.com/watch?v=aTL8Gx6TlCs)


 ---- 
Самое маленькое кино в мире: Мальчик и его атом. 

 [![](https://img.youtube.com/vi/oSCX78-8-q0/0.jpg)](https://www.youtube.com/watch?v=oSCX78-8-q0)


 ---- 
Примеры наноматериалов в природе и "древности": цеолиты,  магниточувствительные бактерии, броня моллюсков, лапы гекконов. 

 [![](https://img.youtube.com/vi/0eo5AUDCFzQ/0.jpg)](https://www.youtube.com/watch?v=0eo5AUDCFzQ)


 ---- 
Научно-популярная телепередача о получении и использовании беспористых тонкопленочных покрытий, свойствах наночастиц, применениях наночастиц в медицине, проблеме транспорта и накопления наночастиц, композиционных наноматериалах для авиации и космоса, лазерно-плазменных методах синтеза.

 [![](https://img.youtube.com/vi/GhvpcJqSKTU/0.jpg)](https://www.youtube.com/watch?v=GhvpcJqSKTU)


 ---- 
# 2. [Электронное строение низкоразмерных структур](02_-_From_atoms_to_nanomaterials/Lecture_and_lab.ipynb) 
В этом видео дается пример простого рассмотрения молекулы водорода. 

 [![](https://img.youtube.com/vi/4vrjwkNFqVg/0.jpg)](https://www.youtube.com/watch?v=4vrjwkNFqVg)


 ---- 
Телепередача о свойствах газообразного и жидкого кислорода, синглетного кислорода и озона

 [![](https://img.youtube.com/vi/vuSplAoifE0/0.jpg)](https://www.youtube.com/watch?v=vuSplAoifE0)


 ---- 
Химия благородных газов.

 [![](https://img.youtube.com/vi/84qTPlAraA4/0.jpg)](https://www.youtube.com/watch?v=84qTPlAraA4)


 ---- 
Квантовая механика и химическая связь: история развития вопроса. 

 [![](https://img.youtube.com/vi/KAA9eTQVQgU/0.jpg)](https://www.youtube.com/watch?v=KAA9eTQVQgU)


 ---- 
Применение теории молекулярных орбиталей для определения электронного строения бутадиена. 

 [![](https://img.youtube.com/vi/xb6V_VUAGGQ/0.jpg)](https://www.youtube.com/watch?v=xb6V_VUAGGQ)


 ---- 
Ароматические соединения, сопряженные системы пи-электронов и правило Хюккеля (4n+2). 

 [![](https://img.youtube.com/vi/hLHl8M9UYoA/0.jpg)](https://www.youtube.com/watch?v=hLHl8M9UYoA)


 ---- 
Рассказ о свойствах однослойных углеродных нанотрубок.

 [![](https://img.youtube.com/vi/Sb1_GxY1L2Q/0.jpg)](https://www.youtube.com/watch?v=Sb1_GxY1L2Q)


 ---- 
# 3. [Кластеры](03_-_Clusters/Lecture.ipynb) 
# 4. [Металлические наночастицы](04_-_Metal_NPs/Lecture.ipynb) 
Пример цитратного коллоидного синтеза наночастиц золота. 

 [![](https://img.youtube.com/vi/urmi99jQSZY/0.jpg)](https://www.youtube.com/watch?v=urmi99jQSZY)


 ---- 
Пример боргидридного коллоидного синтеза наночастиц серебра. 

 [![](https://img.youtube.com/vi/L-GwpAJ9MJ0/0.jpg)](https://www.youtube.com/watch?v=L-GwpAJ9MJ0)


 ---- 
Иллюстрация процесса лазерной абляции на примере золота, серебра, титана, цинка и других металлов. 

 [![](https://img.youtube.com/vi/kOy0yuWpUzU/0.jpg)](https://www.youtube.com/watch?v=kOy0yuWpUzU)


 ---- 
Плазмонные наноструктуры. 

 [![](https://img.youtube.com/vi/YN19Z_IFVwY/0.jpg)](https://www.youtube.com/watch?v=YN19Z_IFVwY)


 ---- 
Нанофотоника. Проблема больших данных и мощностей. Переход от электронике к фотонике, электро и магнитооптике, плазмонных кристаллов, создания миниатюрных источников света — спазеров (SPASER). 

 [![](https://img.youtube.com/vi/r7T1nCG8QSU/0.jpg)](https://www.youtube.com/watch?v=r7T1nCG8QSU)


 ---- 
Диалоги "Управление светом с помощью плазмонов" (Часть 1). 

 [![](https://img.youtube.com/vi/Hdmhcud2RqE/0.jpg)](https://www.youtube.com/watch?v=Hdmhcud2RqE)


 ---- 
Диалоги "Управление светом с помощью плазмонов" (Часть 2). 

 [![](https://img.youtube.com/vi/PQkQpFU9xL4/0.jpg)](https://www.youtube.com/watch?v=PQkQpFU9xL4)


 ---- 
# 5. [Нанокристаллы полупроводниковых материалов](05_-_Semiconductor_NPs/Lecture.ipynb) 
Советский фильм о механизмах люминесценции люминофоров различного химического строения, КПД различных источников излучения, применении люминесценции (легко смотрится на скорости 1.5)

 [![](https://img.youtube.com/vi/WmsqUVizqhk') # 9:/0.jpg)](https://www.youtube.com/watch?v=WmsqUVizqhk') # 9:)


 ---- 
Квантовые точки: размеры, одноэлектронный транзистор. 

 [![](https://img.youtube.com/vi/XkZvKJ3XmZQ/0.jpg)](https://www.youtube.com/watch?v=XkZvKJ3XmZQ)


 ---- 
Рассказ об экситонах, поляритонах и квантовых компьютерах 

 [![](https://img.youtube.com/vi/gNE7-Tw7pzE/0.jpg)](https://www.youtube.com/watch?v=gNE7-Tw7pzE)


 ---- 
Пример синтеза квантовых точек CdSe с использованием триоктилфосфина и триоктилфосфин оксида. 

 [![](https://img.youtube.com/vi/6oDHZtxTEoo/0.jpg)](https://www.youtube.com/watch?v=6oDHZtxTEoo)


 ---- 
Полная процедура синтеза квантовых точек PbS. 

 [![](https://img.youtube.com/vi/1Tco4dS2idw/0.jpg)](https://www.youtube.com/watch?v=1Tco4dS2idw)


 ---- 
Применение квантовых точек в дисплеях (видео 2014 года, такие дисплеи уже производятся). 

 [![](https://img.youtube.com/vi/HLAJnNKgRd8/0.jpg)](https://www.youtube.com/watch?v=HLAJnNKgRd8)


 ---- 
Мини-лекция Р.Б.Васильева по коллоидному синтезу квантовых точек. 

 [![](https://img.youtube.com/vi/RWyDF3nVYos/0.jpg)](https://www.youtube.com/watch?v=RWyDF3nVYos)


 ---- 
# 6. [Наночастицы широкозонных полупроводников и диэлектриков](06_-_Widegap_NPs/Lecture.ipynb) 
Пример автогорения геля. 

 [![](https://img.youtube.com/vi/OFEbiMxKa_8/0.jpg)](https://www.youtube.com/watch?v=OFEbiMxKa_8)


 ---- 
Фотокатализаторы.

 [![](https://img.youtube.com/vi/mylxBckJs0g/0.jpg)](https://www.youtube.com/watch?v=mylxBckJs0g)


 ---- 
Необычная химия наночастиц и рак легких: о поверхности наночастиц диоксида кремния, силикозе и причинах канцерогенности наночастиц

 [![](https://img.youtube.com/vi/8tDIscuI2kY/0.jpg)](https://www.youtube.com/watch?v=8tDIscuI2kY)


 ---- 
Магнитные наночастицы: применение в медицине.

 [![](https://img.youtube.com/vi/iOKDLne5gqo/0.jpg)](https://www.youtube.com/watch?v=iOKDLne5gqo)


 ---- 
Низкоразмерный магнетизм.

 [![](https://img.youtube.com/vi/LIJFfsZbCYU/0.jpg)](https://www.youtube.com/watch?v=LIJFfsZbCYU)


 ---- 
Гигантское магнетосопротивление. 

 [![](https://img.youtube.com/vi/PLD0DdTtUcw/0.jpg)](https://www.youtube.com/watch?v=PLD0DdTtUcw)


 ---- 
Наноалмазы: история Вселенной и применение в квантовой оптике. 

 [![](https://img.youtube.com/vi/Lefp-sW4D-M/0.jpg)](https://www.youtube.com/watch?v=Lefp-sW4D-M)


 ---- 
Наноалмазы: об Арзамасе-16, детонационный синтез алмазов, применение в медицине и однофотонных источниках. 

 [![](https://img.youtube.com/vi/GtaQk2om2ro/0.jpg)](https://www.youtube.com/watch?v=GtaQk2om2ro)


 ---- 
Нанофотоника в алмазах: азотные, кремниевые и германиевые центры окраски в объемных алмазах и наноалмазах. 

 [![](https://img.youtube.com/vi/3SbGBQ30KBs/0.jpg)](https://www.youtube.com/watch?v=3SbGBQ30KBs)


 ---- 
Телепередача о свойствах различных стекол. Получение стекла из песка. Влияние примесей на цвет стекла. Промышленная резка стекла. Важность ровной кромки для механической прочности. Получение и особенности закаленного и многослойного стекол. Взрыв закаленного стекла. Стандартные тесты прочности стекла.

 [![](https://img.youtube.com/vi/l_T9gXYVhi0/0.jpg)](https://www.youtube.com/watch?v=l_T9gXYVhi0)


 ---- 
Телепередача с российского предприятия по росту макроскопических пьезокристаллов лангасита La3Ga5SiO14 и сцинциляционных кристаллов молибдата кальция.

 [![](https://img.youtube.com/vi/XHaFPN-qu1Q/0.jpg)](https://www.youtube.com/watch?v=XHaFPN-qu1Q)


 ---- 
В этом видео 1981 года описываются химические свойства и применение f-элементов: лантаноидов и актиноидов

 [![](https://img.youtube.com/vi/V9Ht1_nWIew/0.jpg)](https://www.youtube.com/watch?v=V9Ht1_nWIew)


 ---- 
Видеорассказ 2018 г. о современном предприятии в Новосибирске по изготовлению наноструктурированной керамики для различных областей применения: протезирования суставов, бронежилетов и керамических подложек для электроники. Подробно описан технический процесс и методы диагностики конечных продуктов. 

 [![](https://img.youtube.com/vi/6_TqTdSbOpA/0.jpg)](https://www.youtube.com/watch?v=6_TqTdSbOpA)


 ---- 
# 7. [Общие свойства наночастиц различной природы](07_-_Properties_of_NPs/Lecture.ipynb) 
На видео ниже представлена иллюстрация броуновского движения. Наблюдаемые вспышки - рассеиваемое наночастицами лазерное излучение.

 [![](https://img.youtube.com/vi/cDcprgWiQEY/0.jpg)](https://www.youtube.com/watch?v=cDcprgWiQEY)


 ---- 
Немой фильм 1937 г. о Броуновском движении

 [![](https://img.youtube.com/vi/7YhAnYiIwSE/0.jpg)](https://www.youtube.com/watch?v=7YhAnYiIwSE)


 ---- 
Метод очистки газа от малых частиц в присутствии электрического поля

 [![](https://img.youtube.com/vi/7YhAnYiIwSE/0.jpg)](https://www.youtube.com/watch?v=7YhAnYiIwSE)


 ---- 
Дисперсия и рассеяние света

 [![](https://img.youtube.com/vi/gW6sMiM3ZDc/0.jpg)](https://www.youtube.com/watch?v=gW6sMiM3ZDc)


 ---- 
**У этого видео пока нет описания.**

 [![](https://img.youtube.com/vi/FvSgyZBZr5s/0.jpg)](https://www.youtube.com/watch?v=FvSgyZBZr5s)


 ---- 
# 8. [Нанопроволоки и наностержни](08_-_1D_Structures/Lecture.ipynb) 
Рассказ о необычных свойствах нанопроволок антимонида индия InSb, квантовании проводимости, введении примесей в нанопроволоки, туннельной проводимости. Далее описываются технологии, связанные с созданием электронных структур, атомном манипулировании, одноатомных электронных устройствахб в частности одноэлектронного транзистора.  

 [![](https://img.youtube.com/vi/sL3DD24bx2k/0.jpg)](https://www.youtube.com/watch?v=sL3DD24bx2k)


 ---- 
Нанотрубки из углерода и нитрида бора. 

 [![](https://img.youtube.com/vi/vPOaam_nLVQ/0.jpg)](https://www.youtube.com/watch?v=vPOaam_nLVQ)


 ---- 
# 9. [Тонкие пленки и покрытия](09_-_2D_Nanomaterials/01_Thin_films.ipynb) 
# 10. [Фотонные кристаллы и многослойные структуры](09_-_2D_Nanomaterials/02_Photonic_crystals.ipynb) 
**У этого видео пока нет описания.**

 [![](https://img.youtube.com/vi/ufppOGpi2gM/0.jpg)](https://www.youtube.com/watch?v=ufppOGpi2gM)


 ---- 
# 11. [Консолидированные наноматериалы и нанокомпозиты](10_-_3D_Nanomaterials/Nanocomposites.ipynb) 
Кристаллическое строение макроскопических металлов и сплавов

 [![](https://img.youtube.com/vi/rqOHFmAAvKo/0.jpg)](https://www.youtube.com/watch?v=rqOHFmAAvKo)


 ---- 
Видео 1982 г. о механике разрушения материалов: влияние трещин, местные напряжения, метод фотоупругости, эпюры напряжений, датчики раскрытия трещины, растяжение образца, критическая нагрузка, критические номинальные напряжения, постояннство коэффициента интенсивности, скорость распространения трещин, расчет материала на прочность, запас прочности определение разрушающих напряжений, проверка прочности, допускаемый размер трещины, допускаемые напряжения. Влияние циклических нагрузок. Методы борьбы с механическими причинами разрушения"  Есть пропуски звука.

 [![](https://img.youtube.com/vi/xrmDF4jukcA') # 20 ми/0.jpg)](https://www.youtube.com/watch?v=xrmDF4jukcA') # 20 ми)


 ---- 
Композиты с использованием BN-нанотрубок: свойства нитрида бора, механические свойства, преимущества в составе полимерных композитов, проблемы когезии и диспергирования. 

 [![](https://img.youtube.com/vi/754Pow_t_qg/0.jpg)](https://www.youtube.com/watch?v=754Pow_t_qg)


 ---- 
# 12. [Самоорганизация и самосборка](10_-_3D_Nanomaterials/Self-organization.ipynb) 
Фильм 1987 г Нелинейные структуры в синергетике Ячейки Бенара Открытые биологические системы Реакция Белоусова-Жаботинского Уравнение диффузии Уравнение синус-Гордона Солитоны Модель типа реакции диффузии Диссипативные структуры

 [![](https://img.youtube.com/vi/XHaFPN-qu1Q/0.jpg)](https://www.youtube.com/watch?v=XHaFPN-qu1Q)


 ---- 
